# Übung 1 Einlesen und Anzeigen der Datensätze

**German Traffic Sign Detection Benchmark**

Detallierte Beschreibung des Datensatzes siehe unter folgendem [Link](http://benchmark.ini.rub.de/?section=gtsdb&subsection=news)

## Imports

In [4]:
# Die folgenden Anweisungen müssen nur ein mal ausgeführt werden
!pip install -U -q ipywidgets
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [1]:
import os

import csv
import wget
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual, widgets

In [2]:
# Testfunktion für ipywidgets: 
# Es soll ein Slider angezeigt werden. Der Wertebereich des Sliders
# soll zwischen -10(min) und 30(max) liegen. 
# Entsprechend der Sliderposition soll ein Ergebniswert angezeigt werden.
def f(x):
    return 3 * x
interact(f, x= 10);

interactive(children=(IntSlider(value=10, description='x', max=30, min=-10), Output()), _dom_classes=('widget-…

## Verwendete  Module / Funktionen

Mache dich mit der Funktionsweise foldender Module / Funktionen vertraut.
Sie können zur Lösung der Aufgaben eingesetzt werden.
- os.path
- os.listdir
- [csv.reader](https://docs.python.org/3/library/csv.html)
- string.split
- np.unique
- list.append
- plt.imread
- plt.imshow
- plt.show
- dict
- zip
- dict(zip())
- sorted
- cv2.rectangle
- cv2.putText
- dict.keys
- dict.values
- enumerate
- range
- interact

## Aufgabe 1 – Einlesen der "Ground Truth"-Textdatei

### Aufgabe 1.a
Lade den Datensatz herunter und entpacke ihn.
Unter folgendem [Link](https://sid.erda.dk/public/archives/ff17dc924eba88d5d01a807357d6614c/published-archive.html) kannst du die Kurzbeschreibung des Datensatzes einsehen.

In [14]:
# Es wird ein Weilchen dauern die Daten herunterzuladen (1,54GB)
# Nach dem erfolgreichen Herunterladen kann dieser Block auskommentiert werden
# url = "https://sid.erda.dk/public/archives/ff17dc924eba88d5d01a807357d6614c/FullIJCNN2013.zip"
# wget.download(url)

In [21]:
# Definiere den Pfad zum heruntergeladenen Datenordner
path_to_data =  r'C:\Users\hqpet\Desktop\BildAuto_Hausaufgabe\UE1\FullIJCNN2013\FullIJCNN2013'
# Prüfe, ob der Pfad existiert / korrekt eingegeben wurde
assert os.path.exists(path_to_data), "Der angegebene Pfad existriert nicht."

In [22]:
# Um hartcodierte Namen in den Funktionen zu vermeiden, definiere an dieser Stelle 
# alle Variablen, die als Eingabeparameter der Funktionen verwendet werden
gt_txt = "gt.txt"

### Aufgabe 1.c
Ermittle die Namen und die Anzahl der Bilder, die in der "Ground Truth"-Textdatei gt.txt nicht annotiert wurden.

Nützliche Funktionen:
- os.path
- [csv.reader](https://docs.python.org/3/library/csv.html)
- string.split
- len
- np.unique
- list.append
- range

In [35]:
def list_not_annotated_images(path_to_data_folder, gt_txt_file):
    """
        Liest Verkehrszeichendaten des German Traffic Sign Detection Benchmarks
        Argumente: Pfad zum heruntergeladenen Datenordner, gt.txt-Datei
        Rückgabe:  Liste mit den Namen der Bilder, die nicht annotiert wurden, 
                   Anzahl nicht annotierter Bilder
    """
    ###   TO DO   ###
    # Definiere den Pfad zur gt.txt
    txt_filepath =  r'C:\Users\hqpet\Desktop\BildAuto_Hausaufgabe\UE1\FullIJCNN2013\FullIJCNN2013\gt.txt'
    
    assert os.path.exists(txt_filepath), "Der angegebene Pfad existriert nicht."
    
    # Definiere eine leere Liste für Bildnamen
    list_img_names = []
    list_missing_names = []
    # Öffne die gt.txt-Datei
    with open(txt_filepath, newline='') as csvfile:
        gt_reader = csvfile
        print("Open file gt.txt: ")
        # Bau eine Schleife, um die Daten Zeile für Zeile einzulesen und list_img_names zu füllen
        for row in gt_reader:
            #print(row.split(";")[0])
            list_img_names.append(row.split(";")[0])
            
    # Entferne doppelte Einträge aus der Liste 
    list_img_names = np.unique(np.array(list_img_names))
    
    # Ermittle, welche Bildnamen fehlen und mache daraus eine Liste 
    filelist = os.listdir(path_to_data) # list all the file names in the path directory
    for file in filelist:
        if file.endswith(".ppm") and file not in list_img_names:
            list_missing_names.append(file)
    print("Saved all unlabled images under list list_missing_names ")
    
    # Ermittle die Anzahl der fehlenden Bildnamen
    number_missing_img = len(list_missing_names)
    
    # Gebe folgendes aus: "In total XYZ images in the data folder are not annotated."
    # Anstelle von XYZ soll die Anzahl der nicht annotierten Bilder ausgegeben werden
    print(f"number_missing_img = {number_missing_img}")
    
    return list_missing_names, number_missing_img

In [36]:
# Rufe die Funktion aus und prüfe, ob alles wie erwartet funktioniert
missing_img_list, missing_img_number = list_not_annotated_images(path_to_data, gt_txt)

Open file gt.txt: 
Saved all unlabled images under list list_missing_names 
number_missing_img = 159


### Aufgabe 1.d
Schreibe eine Funktion, die die gt.txt-Datei einliest und drei Listen zurückgibt:
- die Liste mit relativen Bildpfaden (strings),
- die Liste mit ClassIDs (integers)
- die Liste mit ROI-Koordinaten (integers)

Einzelne Schritte kannst du aus list_not_annotated_images-Funktion übernehmen.

In [37]:
def read_txt(path_to_data_folder, gt_txt_file):
    """
        Liest Verkehrszeichendaten des German Traffic Sign Detection Benchmarks
        Argumente: Pfad zum heruntergeladenen Datenordner
        Rückgabe:  Liste mit relativen Bildpfaden, Liste mit ClassIDs, Liste mit ROI-Koordinaten
    """
    ###   TO DO   ###
    # Definiere den Pfad zur gt.txt
    txt_filepath = 'C:/Users/hqpet/Desktop/BildAuto_Hausaufgabe/UE1/FullIJCNN2013/FullIJCNN2013/mapping.csv'
    
    # Prüfe, ob der Pfad existiert / korrekt eingegeben wurde
    assert os.path.exists(txt_filepath), "Der angegebene Pfad existriert nicht."
    
    # Definiere leere Listen
    # Liste für Bildpfade
    img_paths_list = []
    
    # Liste für Class_IDs
    class_ids_list =[]
    # Liste für ROIs
    rois_list = []
    
    # Öffne die gt.txt-Datei und ergänze den Code, um die entsprechenden Listen zu füllen
    with open(txt_filepath, newline='') as csvfile:
        gt_reader = csvfile
        
        for row in gt_reader:
            infolist = row.split(";")
            img_paths_list.append(path_to_data_folder + "/" +infolist[0])
            class_ids_list.append(int(infolist[-1][:-1]))
            rois_list.append([int(i) for i in infolist[1:-1]])
    
        return(img_paths_list, class_ids_list, rois_list)

In [38]:
ppm_file_paths, class_ids, rois = read_txt(path_to_data, gt_txt)

ValueError: invalid literal for int() with base 10: '\ufeffClassID,Traffic Sign Name\r'

In [34]:
# Teste, ob die Bilder sich anzeigen lassen
# Nutze hier die Listen ppm_filenames, class_ids, die du mit read_txt berechnet hast
img = plt.imread(ppm_file_paths[80])
print("Bildgröße", img.shape)
print("Class ID: ", class_ids[80])
plt.imshow(img)
plt.show()

NameError: name 'ppm_file_paths' is not defined

## Aufgabe 2 – Mapping ClassID – Bezeichnung der Verkehrsschilder
Generiere eine csv-Mapping-Datei aus ReadMe.txt. Nutze dafür alle dir zur Verfügung stehenden Mittel. Python Code ist für diese Aufgabe kein Muss. Die Datei soll das Mapping von Zahl zur Verkehrszeichenbezeichnung enthalten.
Schreibe eine Funktion, die zwei Rückgabewerte zurückgibt:
- pandas-DataFrame aus der generierten csv-Datei und
- eine Dictionary (dict) mit ClassIDs als *keys* und Verkehrsschilderbezeichnungen als *values*.

Nützliche Module / Funktionen:
- pd.read_csv
- os.path
- dict(zip())

In [28]:
###   TO DO   ###
# Generiere eine csv-Mapping-Datei und lege sie in den Datenordner ab
# Definiere die Variable csv_mapping, z.B.: 'tf_signs_mapping.csv' (entspricht dem Namen der generierten Datei)
csv_mapping = "FullIJCNN2013/tf_signs_mapping1.csv"

In [29]:
def map_int_to_sign_name(path_to_data_folder, csv_mapping_file):
    """
    Ordnet int-Zahl dem Schildnamen zu
    Argumente: Pfad zum Datenordner, Name der csv-Datei
    Rückgabe:  pandas-DataFrame aus der generierten csv-Datei und 
              eine Dictionary (dict) mit ClassIDs als keys und Traffic Sign Names als values
    """
    
    ###   TO DO   ###
    csv_path = csv_mapping_file
    
    assert os.path.exists(csv_path), "Der angegebene Pfad existriert nicht."
    # Lese die csv-Datei als DataFrame ein
    df = pd.read_csv(csv_path, delimiter = ",",header = 0)
    # dict_mapping-Variable soll eine Dictionary (dict) sein, mit ClassIDs als keys und Traffic Sign Names als values
    #dict_mapping = dict(zip(df["ID"], df["Name"]))
    dict_mapping = df.to_dict()["Name"]
    print(dict_mapping[1])
    return dict_mapping, df

In [30]:
dict_mapping, df_map = map_int_to_sign_name(path_to_data, csv_mapping)

AssertionError: Der angegebene Pfad existriert nicht.

In [31]:
# Übersicht der Klassen
df_map.style.hide_index()

NameError: name 'df_map' is not defined

## Aufgabe 3 – Visualisierung der Verkehrszeichen
Schreibe eine Funktion, die ein Bild pro Klasse ausgibt. Verwende dafür die Bilder in den Unterordnern, die 
bereits nach **ClassID** genannt sind, um die Klassenzuordnung herzustellen. Für das Mapping verwende Variable **dict_mapping**, die in Aufgabe 2 berechnet wurde.

Nützliche Module / Funktionen:
- interact
- os.path
- list.append

In [79]:
def one_image_per_class(path_to_data_folder):
    """
    Gibt eine Liste mit je einem Bildpfad pro Klasse zurück
    Argumente: Pfad zum Datenordner
    Rückgabe: Liste mit Bildpfaden
    """
    ###   TO DO   ###
    # Definiere eine leere Liste für Bildpfade
    img_paths = []
    
    # Generiere eine Liste mit den Namen der Unterordner. Python os-Modul kann hier nützlich sein
    subfolders_names = next(os.walk(path_to_data_folder))[1]
    subfolders_paths = []
    for folder in subfolders_names:
        subfolders_paths.append(path_to_data_folder + "/" + folder)
    # Iteriere über die Unterordner-Pfade und speichere je ein Bildpfad aus den Unterordnern 
    for path in subfolders_paths:
        assert os.path.exists(path), "Der angegebene Pfad existriert nicht1."
        # Generiere einen gültigen Bildpfad
        img_path = path + "/00001.ppm"
        
        assert os.path.exists(img_path), "Der angegebene Pfad existriert nicht2."
        
        # Füge jeden Bildpfad der Liste img_paths hinzu
        img_paths.append(img_path)
        
    return img_paths

In [80]:
img_paths_43 = one_image_per_class(path_to_data)

In [81]:
def show_img_tr_sign(idx):
    ###   TO DO   ###
    # Definiere eine print()- Funktion deren Ausgabe folgende Form hat:
    # ClassID XY: dazu passende Bezeichnung
    # z.B.: ClassID 25:  construction (danger)
    print(f"ClassID {idx}: {dict_mapping[idx]}")
    
    plt.figure(figsize=(6,6))
    img = plt.imread(img_paths_43[idx])
    plt.imshow(img)
    plt.show()

In [78]:
# Nutze interaktive Anzeige, um die Bilder anzusehen
interact(show_img_tr_sign, idx=widgets.IntSlider(min=0,max=len(img_paths_43)-1,step=1, value=0));

interactive(children=(IntSlider(value=0, description='idx', max=42), Output()), _dom_classes=('widget-interact…

## Aufgabe 4 – Anzeige der ROIs (Regions of Interest)
Schreibe eine Funktion, um innerhalb der angezeigten Bilder die Verkehrsschilder zu markieren. Verwende dafür die Bilder im Hauptordner und die dazugehörige gt.txt. Bedenke, dass jedes Bild nur einmal angezeigt werden soll. 

Nützliche Module / Funktionen:
- dict(zip())
- cv2.rectangle
- cv2.putText

In [182]:
def calc_rois(path_to_data_folder, csv_mapping_file, gt_txt_file):
    """
    Zeichnet ROIs und deren Bezeichnungen in die Bilder ein
    Argumente: Pfad zum heruntergeladenen Datenordner, Dateinamen
    Rückgabe:  Liste mit Bildern 
    """
    # Die Funktionen map_int_to_sign_name und read_txt sollten bereits implementiert sein  
    map_tr_sing_int, df_map = map_int_to_sign_name(path_to_data_folder, csv_mapping_file) #dict + df
    ppm_filenames, class_ids, rois = read_txt(path_to_data_folder, gt_txt_file) 
    
    # rois[x] = [x1,y1,x2,y2] 
    ###   TO DO   ###
    # Definiere eine leere Liste für die Speicherung von Bildern
    data = []
    # Definiere ein leeres Dictionary
    data_dict ={}
    
    curr_path = ""
    img = None
    counter_identical_path = 0
    # Bevor du weitermachst, versuche zu verstehen, was in if- und else-Blöcken der Schleife passiert
    # Antwort: ein Bild kann mehrere Bild beinhalten, sie alle sollen makiert werden.
    for idx, file_path in enumerate(ppm_filenames):
        if curr_path != file_path: 
            curr_path = file_path
            # Lese ein Bild ein
            img = plt.imread(file_path)
            counter_identical_path = 1
            # Definiere Koordinaten für die Positionierung des Textfeldes mit Beschreibung des Verkehrszeichens
            initial_x_coordinate = rois[idx][0]
            initial_y_coordinate = rois[idx][1]
            # Bei dieser Variable handelt es sich um org-Parameter der cv2.putText
            org_id_meaning = (initial_x_coordinate, initial_y_coordinate)
        else:
            counter_identical_path += 1
            # Passe die x-Koordinate für jedes weitere Verkehrszeichen an
            # Folgende Zeilen sind eventuell optional 
            # Es hängt davon ab, wo du die Bezeichnung positionierst
            initial_x_coordinate = rois[idx][0]
            initial_y_coordinate = rois[idx][1]
            org_id_meaning = (initial_x_coordinate, initial_y_coordinate)
        
        # Berechne Koordinaten des Rechtecks, benutze dafür die Variable rois
        print(rois[idx])
        point1 = (rois[idx][0],rois[idx][1])
        point2 = (rois[idx][2],rois[idx][3])
        
        # Zeichne das Rechteck mit Hilfe der berechneten Koordinaten in das Bild (cv2.rectangle) ein
        print(point1, point2, (0,0,0),2)
        img = cv2.rectangle(img, point1, point2, (255,0,0),2)
        
        # Ermittle Koordinaten für das Textfeld
        org = (rois[idx][2] + 10, rois[idx][1] + 20)
        # Speichere Verkehrszeichen-ID als string
        text = str(class_ids[idx])
        # Nutze cv2.putText um die Verkehrszeichen-ID neben dem entsprechenden Verkehrszeichen zu positionieren
        # font
        font = cv2.FONT_HERSHEY_SIMPLEX
        # fontScale
        fontScale = 1
        # Blue color in BGR
        color = (255, 0, 0)
        # Line thickness of 2 px
        thickness = 3
        img = cv2.putText(img, text, org, font, fontScale, color, thickness)
        # Speichere Verkehrszeichen-ID mit der dazugehörigen Bezeichnung als string
        text_id_meaning = map_tr_sing_int[class_ids[idx]]
        
        
        # Nutze cv2.putText um text_id_meaning im Bild zu positionieren
        # font
        font2 = cv2.FONT_HERSHEY_SIMPLEX
        # fontScale
        fontScale2 = 0.7
        # Blue color in BGR
        color2 = (255, 0, 0)
        # Line thickness of 2 px
        thickness2 = 2
        org2 = (rois[idx][2]+50, rois[idx][1] + 20)
        img = cv2.putText(img, text_id_meaning, org2, font2, fontScale2, color2, thickness2)
        
        
        
        # Füge jedes Bild der data-Liste hinzu
        data.append(img)
    
    # Benutze die Verkettung dict(zip()). Die Dictionary soll Pfadnamen als keys und Bilder als Dictionaries enthalten 
    data_dict = dict(zip(ppm_filenames, data))
    # Schpeichere die Bilder aus data_dict in die Liste
    data_list = data
    
    return data_list

In [183]:
data = calc_rois(path_to_data, csv_mapping, gt_txt)                                     

 speed limit 30 (prohibitory)
[774, 411, 815, 446]
(774, 411) (815, 446) (0, 0, 0) 2
[983, 388, 1024, 432]
(983, 388) (1024, 432) (0, 0, 0) 2
[386, 494, 442, 552]
(386, 494) (442, 552) (0, 0, 0) 2
[973, 335, 1031, 390]
(973, 335) (1031, 390) (0, 0, 0) 2
[892, 476, 1006, 592]
(892, 476) (1006, 592) (0, 0, 0) 2
[742, 443, 765, 466]
(742, 443) (765, 466) (0, 0, 0) 2
[742, 466, 764, 489]
(742, 466) (764, 489) (0, 0, 0) 2
[737, 412, 769, 443]
(737, 412) (769, 443) (0, 0, 0) 2
[898, 342, 967, 409]
(898, 342) (967, 409) (0, 0, 0) 2
[906, 407, 955, 459]
(906, 407) (955, 459) (0, 0, 0) 2
[1172, 164, 1284, 278]
(1172, 164) (1284, 278) (0, 0, 0) 2
[926, 350, 989, 414]
(926, 350) (989, 414) (0, 0, 0) 2
[825, 406, 864, 445]
(825, 406) (864, 445) (0, 0, 0) 2
[785, 460, 811, 486]
(785, 460) (811, 486) (0, 0, 0) 2
[779, 424, 816, 461]
(779, 424) (816, 461) (0, 0, 0) 2
[925, 466, 949, 490]
(925, 466) (949, 490) (0, 0, 0) 2
[1193, 358, 1269, 436]
(1193, 358) (1269, 436) (0, 0, 0) 2
[720, 338, 770, 398]


[847, 349, 897, 397]
(847, 349) (897, 397) (0, 0, 0) 2
[767, 462, 808, 499]
(767, 462) (808, 499) (0, 0, 0) 2
[695, 461, 730, 492]
(695, 461) (730, 492) (0, 0, 0) 2
[256, 438, 303, 484]
(256, 438) (303, 484) (0, 0, 0) 2
[958, 378, 1024, 440]
(958, 378) (1024, 440) (0, 0, 0) 2
[902, 369, 985, 443]
(902, 369) (985, 443) (0, 0, 0) 2
[486, 485, 540, 537]
(486, 485) (540, 537) (0, 0, 0) 2
[490, 536, 529, 576]
(490, 536) (529, 576) (0, 0, 0) 2
[990, 484, 1020, 513]
(990, 484) (1020, 513) (0, 0, 0) 2
[929, 489, 954, 513]
(929, 489) (954, 513) (0, 0, 0) 2
[1091, 465, 1123, 497]
(1091, 465) (1123, 497) (0, 0, 0) 2
[350, 503, 382, 535]
(350, 503) (382, 535) (0, 0, 0) 2
[366, 473, 399, 508]
(366, 473) (399, 508) (0, 0, 0) 2
[1186, 475, 1225, 514]
(1186, 475) (1225, 514) (0, 0, 0) 2
[351, 413, 419, 480]
(351, 413) (419, 480) (0, 0, 0) 2
[1169, 421, 1231, 484]
(1169, 421) (1231, 484) (0, 0, 0) 2
[850, 218, 950, 317]
(850, 218) (950, 317) (0, 0, 0) 2
[330, 523, 351, 545]
(330, 523) (351, 545) (0, 0,

[1011, 385, 1043, 418]
(1011, 385) (1043, 418) (0, 0, 0) 2
[667, 410, 705, 445]
(667, 410) (705, 445) (0, 0, 0) 2
[293, 427, 315, 450]
(293, 427) (315, 450) (0, 0, 0) 2
[721, 381, 749, 408]
(721, 381) (749, 408) (0, 0, 0) 2
[756, 402, 782, 430]
(756, 402) (782, 430) (0, 0, 0) 2
[1185, 130, 1295, 236]
(1185, 130) (1295, 236) (0, 0, 0) 2
[1214, 241, 1288, 312]
(1214, 241) (1288, 312) (0, 0, 0) 2
[784, 480, 804, 500]
(784, 480) (804, 500) (0, 0, 0) 2
[312, 442, 337, 467]
(312, 442) (337, 467) (0, 0, 0) 2
[886, 446, 911, 472]
(886, 446) (911, 472) (0, 0, 0) 2
[971, 267, 1031, 328]
(971, 267) (1031, 328) (0, 0, 0) 2
[808, 437, 827, 456]
(808, 437) (827, 456) (0, 0, 0) 2
[1080, 270, 1155, 340]
(1080, 270) (1155, 340) (0, 0, 0) 2
[1089, 345, 1144, 401]
(1089, 345) (1144, 401) (0, 0, 0) 2
[1127, 285, 1168, 326]
(1127, 285) (1168, 326) (0, 0, 0) 2
[988, 420, 1042, 470]
(988, 420) (1042, 470) (0, 0, 0) 2
[985, 354, 1022, 390]
(985, 354) (1022, 390) (0, 0, 0) 2
[816, 334, 858, 374]
(816, 334) (85

[1213, 398, 1247, 434]
(1213, 398) (1247, 434) (0, 0, 0) 2
[396, 451, 429, 485]
(396, 451) (429, 485) (0, 0, 0) 2
[1267, 373, 1310, 419]
(1267, 373) (1310, 419) (0, 0, 0) 2
[172, 414, 213, 462]
(172, 414) (213, 462) (0, 0, 0) 2
[693, 413, 719, 437]
(693, 413) (719, 437) (0, 0, 0) 2
[696, 436, 719, 458]
(696, 436) (719, 458) (0, 0, 0) 2
[994, 395, 1022, 420]
(994, 395) (1022, 420) (0, 0, 0) 2
[995, 421, 1020, 445]
(995, 421) (1020, 445) (0, 0, 0) 2
[1122, 378, 1166, 421]
(1122, 378) (1166, 421) (0, 0, 0) 2
[631, 451, 657, 477]
(631, 451) (657, 477) (0, 0, 0) 2
[779, 481, 800, 502]
(779, 481) (800, 502) (0, 0, 0) 2
[1201, 403, 1248, 450]
(1201, 403) (1248, 450) (0, 0, 0) 2
[569, 436, 612, 481]
(569, 436) (612, 481) (0, 0, 0) 2
[175, 438, 202, 466]
(175, 438) (202, 466) (0, 0, 0) 2
[921, 403, 951, 434]
(921, 403) (951, 434) (0, 0, 0) 2
[1166, 327, 1209, 370]
(1166, 327) (1209, 370) (0, 0, 0) 2
[338, 406, 381, 449]
(338, 406) (381, 449) (0, 0, 0) 2
[398, 437, 425, 464]
(398, 437) (425, 464

[794, 531, 816, 553]
(794, 531) (816, 553) (0, 0, 0) 2
[533, 559, 554, 580]
(533, 559) (554, 580) (0, 0, 0) 2
[841, 556, 862, 577]
(841, 556) (862, 577) (0, 0, 0) 2
[1194, 504, 1251, 564]
(1194, 504) (1251, 564) (0, 0, 0) 2
[1243, 492, 1324, 562]
(1243, 492) (1324, 562) (0, 0, 0) 2
[1063, 531, 1112, 580]
(1063, 531) (1112, 580) (0, 0, 0) 2
[817, 550, 847, 578]
(817, 550) (847, 578) (0, 0, 0) 2
[823, 579, 843, 598]
(823, 579) (843, 598) (0, 0, 0) 2
[470, 551, 502, 578]
(470, 551) (502, 578) (0, 0, 0) 2
[477, 578, 497, 596]
(477, 578) (497, 596) (0, 0, 0) 2
[379, 528, 427, 573]
(379, 528) (427, 573) (0, 0, 0) 2
[964, 531, 1009, 573]
(964, 531) (1009, 573) (0, 0, 0) 2
[389, 569, 419, 600]
(389, 569) (419, 600) (0, 0, 0) 2
[972, 572, 1001, 601]
(972, 572) (1001, 601) (0, 0, 0) 2
[1021, 498, 1066, 544]
(1021, 498) (1066, 544) (0, 0, 0) 2
[213, 519, 254, 561]
(213, 519) (254, 561) (0, 0, 0) 2
[401, 507, 441, 544]
(401, 507) (441, 544) (0, 0, 0) 2
[767, 497, 803, 530]
(767, 497) (803, 530) (0

[731, 336, 764, 370]
(731, 336) (764, 370) (0, 0, 0) 2
[655, 395, 689, 429]
(655, 395) (689, 429) (0, 0, 0) 2
[124, 419, 156, 451]
(124, 419) (156, 451) (0, 0, 0) 2
[461, 489, 481, 508]
(461, 489) (481, 508) (0, 0, 0) 2
[1050, 409, 1109, 460]
(1050, 409) (1109, 460) (0, 0, 0) 2
[881, 416, 914, 449]
(881, 416) (914, 449) (0, 0, 0) 2
[1287, 361, 1308, 384]
(1287, 361) (1308, 384) (0, 0, 0) 2
[403, 474, 435, 506]
(403, 474) (435, 506) (0, 0, 0) 2
[584, 510, 608, 534]
(584, 510) (608, 534) (0, 0, 0) 2
[700, 454, 722, 476]
(700, 454) (722, 476) (0, 0, 0) 2
[82, 450, 145, 508]
(82, 450) (145, 508) (0, 0, 0) 2
[1268, 555, 1299, 586]
(1268, 555) (1299, 586) (0, 0, 0) 2
[443, 543, 474, 574]
(443, 543) (474, 574) (0, 0, 0) 2
[361, 445, 417, 500]
(361, 445) (417, 500) (0, 0, 0) 2
[365, 482, 437, 546]
(365, 482) (437, 546) (0, 0, 0) 2
[167, 511, 206, 550]
(167, 511) (206, 550) (0, 0, 0) 2
[846, 501, 881, 535]
(846, 501) (881, 535) (0, 0, 0) 2
[853, 501, 887, 535]
(853, 501) (887, 535) (0, 0, 0) 2


[1135, 492, 1181, 537]
(1135, 492) (1181, 537) (0, 0, 0) 2
[235, 469, 281, 515]
(235, 469) (281, 515) (0, 0, 0) 2
[236, 515, 280, 561]
(236, 515) (280, 561) (0, 0, 0) 2
[1138, 537, 1182, 579]
(1138, 537) (1182, 579) (0, 0, 0) 2
[298, 489, 345, 538]
(298, 489) (345, 538) (0, 0, 0) 2
[1148, 506, 1196, 553]
(1148, 506) (1196, 553) (0, 0, 0) 2
[301, 551, 346, 596]
(301, 551) (346, 596) (0, 0, 0) 2
[1153, 546, 1200, 593]
(1153, 546) (1200, 593) (0, 0, 0) 2
[725, 568, 755, 596]
(725, 568) (755, 596) (0, 0, 0) 2
[934, 448, 979, 494]
(934, 448) (979, 494) (0, 0, 0) 2
[728, 593, 767, 632]
(728, 593) (767, 632) (0, 0, 0) 2
[1084, 294, 1208, 418]
(1084, 294) (1208, 418) (0, 0, 0) 2
[852, 518, 883, 549]
(852, 518) (883, 549) (0, 0, 0) 2
[563, 595, 593, 625]
(563, 595) (593, 625) (0, 0, 0) 2
[881, 524, 931, 574]
(881, 524) (931, 574) (0, 0, 0) 2
[982, 521, 1014, 553]
(982, 521) (1014, 553) (0, 0, 0) 2
[374, 515, 406, 547]
(374, 515) (406, 547) (0, 0, 0) 2
[376, 548, 407, 579]
(376, 548) (407, 579) 

[1101, 389, 1171, 452]
(1101, 389) (1171, 452) (0, 0, 0) 2
[590, 470, 610, 488]
(590, 470) (610, 488) (0, 0, 0) 2
[876, 450, 896, 469]
(876, 450) (896, 469) (0, 0, 0) 2
[60, 405, 129, 475]
(60, 405) (129, 475) (0, 0, 0) 2
[1233, 317, 1310, 391]
(1233, 317) (1310, 391) (0, 0, 0) 2
[569, 456, 590, 477]
(569, 456) (590, 477) (0, 0, 0) 2
[1030, 441, 1050, 461]
(1030, 441) (1050, 461) (0, 0, 0) 2
[375, 407, 411, 443]
(375, 407) (411, 443) (0, 0, 0) 2
[1273, 381, 1313, 421]
(1273, 381) (1313, 421) (0, 0, 0) 2
[937, 406, 970, 439]
(937, 406) (970, 439) (0, 0, 0) 2
[318, 416, 351, 449]
(318, 416) (351, 449) (0, 0, 0) 2
[1233, 387, 1298, 450]
(1233, 387) (1298, 450) (0, 0, 0) 2
[466, 469, 491, 494]
(466, 469) (491, 494) (0, 0, 0) 2
[776, 477, 796, 497]
(776, 477) (796, 497) (0, 0, 0) 2
[301, 535, 341, 575]
(301, 535) (341, 575) (0, 0, 0) 2
[988, 506, 1029, 547]
(988, 506) (1029, 547) (0, 0, 0) 2
[1131, 561, 1168, 598]
(1131, 561) (1168, 598) (0, 0, 0) 2
[478, 546, 513, 581]
(478, 546) (513, 581

In [184]:
def show_img(idx):
    '''
    Helper-Funktion, die als erstes Parameter bei interact eingesetzt wird
    '''
    plt.figure(figsize=(16,8))
    plt.imshow(data[idx])
    plt.show()

In [185]:
interact(show_img, idx=widgets.IntSlider(min=0,max=len(data),step=1, value=0));

interactive(children=(IntSlider(value=0, description='idx', max=1213), Output()), _dom_classes=('widget-intera…

## GESCHAFFT !!!